In [2]:
import csv # csv 파일 처리 모듈

In [6]:
with open('data-files/seoul-temperature.csv', 'rt', encoding='utf-8') as f:
    reader = csv.reader(f) # csv 파일 처리기 만들기

    next(reader) # 한 행을 일고 다음 행으로 이동 (헤더를 읽어서 버리기)
    lines = []
    for line in reader:
        lines.append(line)

In [9]:
len(lines), lines[:5]

(40910,
 [['1907-10-01', '108', '13.5', '7.9', '20.7'],
  ['1907-10-02', '108', '16.2', '7.9', '22'],
  ['1907-10-03', '108', '16.2', '13.1', '21.3'],
  ['1907-10-04', '108', '16.5', '11.2', '22'],
  ['1907-10-05', '108', '17.6', '10.9', '25.4']])

In [23]:
# map(함수, 데이터) -> 데이터의 각 행에 함수를 적용해서 결과를 이터레이터로 반환
def to_float(x): # 'value'
    try:
        fx = float(x)
    except:
        fx = -1000
    return fx

data = list( filter(lambda line: len(line) == 5, lines) ) # 각 행별로 데이터의 갯수가 5개인 행만 포함
# list(  map(lambda line: to_float(line[4]), data) )
data = list( map( lambda line: line[:2] + \
                               list( map( lambda x: to_float(x), line[2:] ) ), \
                  data ) )

In [24]:
len(data), data[:3]

(40909,
 [['1907-10-01', '108', 13.5, 7.9, 20.7],
  ['1907-10-02', '108', 16.2, 7.9, 22.0],
  ['1907-10-03', '108', 16.2, 13.1, 21.3]])

In [25]:
# 가장 더웠던 날 ( 최고 기온이 가장 높았던 날) 및 온도

max_temperature = -1000
max_date = ''

for row in data:
    if row[4] > max_temperature:
        max_temperature = row[4] # 그날의 최고 온도
        max_date = row[0] # 날짜 

print(max_date, max_temperature)

2018-08-01 39.6


In [27]:
# 출생 이후 생일 중에서 가장 더웠던 날

birth_year = '1995'
birth_date = '03-01'
max_temperature = -1000
max_date = ''

for row in data:
    year = row[0][:4]
    date = row[0][5:]

    if birth_year <= year and birth_date == date: #출생 이후 생일 중에서
        if row[4] > max_temperature:
            max_temperature = row[4] # 그날의 최고 온도
            max_date = row[0] # 날짜 

print(max_date, max_temperature)

2007-03-01 15.3


In [30]:
# 월별로 데이터 구성 1 (월을 key로 하는 딕셔너리로 구성) --> { '01': [], '02: []}

data_by_month = {}

for idx, row in enumerate(data):
    month = row[0][5:7] # '2000-12-31' -> '12'
    # print(month)
    # if idx == 5: break
    if month in data_by_month:
        data_by_month[month].append(row)
    else:
        data_by_month[month] = [row]

data_by_month['01'][:5]

[['1908-01-01', '108', -5.9, -9.1, -1.3],
 ['1908-01-02', '108', -7.7, -13.9, -3.0],
 ['1908-01-03', '108', -6.2, -10.8, -2.9],
 ['1908-01-04', '108', -7.5, -12.8, -1.7],
 ['1908-01-05', '108', -4.8, -11.3, -0.1]]

In [31]:
# 월별로 데이터 구성 (월을 key로 하는 딕셔너리로 구성) --> { '01': [], '02: []}

from collections import defaultdict # 딕셔너리에 키가 없으면 자동으로 만들고 다음 처리 수행

data_by_month = defaultdict(list)

for idx, row in enumerate(data):
    month = row[0][5:7] # '2000-12-31' -> '12'
    data_by_month[month].append(row)
    
data_by_month['01'][:5]

[['1908-01-01', '108', -5.9, -9.1, -1.3],
 ['1908-01-02', '108', -7.7, -13.9, -3.0],
 ['1908-01-03', '108', -6.2, -10.8, -2.9],
 ['1908-01-04', '108', -7.5, -12.8, -1.7],
 ['1908-01-05', '108', -4.8, -11.3, -0.1]]

In [45]:
# [ true일때 선택되는 값 ] if 조건식 else [ false일 때 선택되는 값 ]
10 if len('hello') > 4 else 20

10

In [48]:
def mean_temperature_by_month(month, t='max'):

    month = "{0:02d}".format(month) # 8 -> "08"

    # 'mean' -> 2, 'min' -> 3, 'max' -> 4
    # idx = 2
    # if t == 'max':
    #     idx = 4
    # elif t == 'min':
    #     idx = 3
    # else:
    #     idx = 2

    idx = 2 if t == 'mean' else 3 if t == 'min' else 4

    total, cnt = 0, 0
    for row in data:
        if month == row[0][5:7]:
            total += row[idx]
            cnt += 1
    return total / cnt

def mean_temperature_by_month2(month, t='max'):

    month = "{0:02d}".format(month) # 8 -> "08"
    idx = 2 if t == 'mean' else 3 if t == 'min' else 4
    total = 0
    for row in data_by_month[month]:
        total += row[idx]

    return total / len(data_by_month[month])


mean_temperature_by_month2(8, t='mean')

16.31996512641674

In [54]:
# 1년 중 일교차가 가장 큰 달 (max - min) 평균

gap_by_month = []
for month in range(1, 13):
    mx = mean_temperature_by_month2(month, 'max')
    mn = mean_temperature_by_month2(month, 'min')
    gap_by_month.append(( month, (mx - mn) ))

# print( gap_by_month )
gap_by_month.sort(key=lambda x: x[1], reverse=True)
gap_by_month

[(5, 10.664141705069124),
 (4, 10.63273809523806),
 (10, 10.318184413359885),
 (6, 9.576666666666668),
 (3, 9.408006912442357),
 (9, 9.240596125186277),
 (11, 9.236428571428517),
 (2, 8.696334913112256),
 (1, 8.273041474654445),
 (12, 8.10822548173708),
 (8, 7.887852368497473),
 (7, 7.244210829493063)]